# YOLOv3

This notebook details how the YOLOv3 model was trained 4-folds, benchmarked and evaluated

install necessary libraries

In [ ]:
import shutil
from google.colab import drive
import os
from ultralytics import YOLO
from pathlib import Path
import torch
from tqdm import tqdm
import numpy as np
from sklearn.metrics import precision_recall_curve, average_precision_score, confusion_matrix, f1_score
import time


Mount google drive to access data stored.

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


install Ultralytics framework

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.4/799.4 kB 18.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

initialise yolov3 model

In [ ]:
model = YOLO('yolov3.yaml')

In [ ]:
# Hardcoded values
ds_yamls = [
    Path('/content/drive/MyDrive/Combined Dataset3/2024-07-07_5-Fold_Cross-val/split_1/split_1_dataset.yaml'),
    Path('/content/drive/MyDrive/Combined Dataset3/2024-07-07_5-Fold_Cross-val/split_2/split_2_dataset.yaml'),
    Path('/content/drive/MyDrive/Combined Dataset3/2024-07-07_5-Fold_Cross-val/split_3/split_3_dataset.yaml'),
    Path('/content/drive/MyDrive/Combined Dataset3/2024-07-07_5-Fold_Cross-val/split_4/split_4_dataset.yaml')
]

Define number of k-fold spllits

In [ ]:
ksplit = 4

In [ ]:
results = {}

batch = 16
project = "kfold"
epochs = 100
# Define early stopping parameters
early_stopping_params = {
    'patience': 10,  # Number of epochs to wait after last improvement
}

for k in range(ksplit):
    dataset_yaml = ds_yamls[k]
    model.train(data=dataset_yaml, patience=early_stopping_params['patience'], epochs=epochs, batch=batch, project=project)  # include train arguments
    results[k] = model.metrics  # save output metrics for further analysis

Ultralytics YOLOv8.2.50 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov3.yaml, data=/content/drive/MyDrive/Combined Dataset3/2024-07-07_5-Fold_Cross-val/split_1/split_1_dataset.yaml, epochs=100, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=kfold, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False

100%|██████████| 755k/755k [00:00<00:00, 132MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 1]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     37056  ultralytics.nn.modules.block.Bottleneck      [64, 64]                      
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    295680  ultralytics.nn.modules.block.Bottleneck      [128, 128]                    
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  8   4724736  ultralytics.nn.modules.block.Bottleneck      [256, 256]                    
  7                  -1  1   1180672  ultralytics

100%|██████████| 6.25M/6.25M [00:00<00:00, 259MB/s]
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


AMP: checks passed ✅


...

                   all        405       1684      0.971      0.898       0.96      0.915

100 epochs completed in 2.409 hours.
Optimizer stripped from kfold/train222/weights/last.pt, 207.7MB
Optimizer stripped from kfold/train222/weights/best.pt, 207.7MB

Validating kfold/train222/weights/best.pt...
Ultralytics YOLOv8.2.50 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv3 summary (fused): 226 layers, 103665011 parameters, 0 gradients, 282.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:07<00:00,  1.77it/s]


                   all        405       1684      0.978      0.893      0.959      0.915
Speed: 0.1ms preprocess, 9.6ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to kfold/train222
Ultralytics YOLOv8.2.50 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov3.yaml, data=/content/drive/MyDrive/Combined Dataset3/2024-07-07_5-Fold_Cross-val/split_4/split_4_dataset.yaml, epochs=100, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=kfold, name=train2222, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=Fals

train: Scanning /content/drive/MyDrive/Combined Dataset3/2024-07-07_5-Fold_Cross-val/split_4/train/labels... 29 images, 0 backgrounds, 0 corrupt:   2%|▏         | 29/1621 [00:08<07:46,  3.41it/s]


KeyboardInterrupt: 

saving training results to a folder

In [ ]:

# Source and destination paths
source_folder = '/content/kfold'
destination_folder = '/content/drive/MyDrive/Yolov3_runs/runs_100epochs-3-fold'

# Copy the folder and its contents
shutil.copytree(source_folder, destination_folder)

'/content/drive/MyDrive/Yolov3_runs/runs_100epochs-3-fold'

training an extra fold to compensate lack of performance on horizontal images by applying rotations to images

In [ ]:

# Load a model
model = YOLO('/content/drive/MyDrive/Yolov3_runs/runs_100epochs-3-fold/train222/weights/last.pt')  # build a new model from scratch

# Define early stopping parameters
early_stopping_params = {
    'patience': 10,  # Number of epochs to wait after last improvement
    'delta': 0.01  # Minimum change to qualify as improvement
}

# Use the model with early stopping
results = model.train(data=os.path.join('/content/drive/MyDrive/Combined Dataset3/2024-07-07_5-Fold_Cross-val/split_4/', 'split_4_dataset.yaml'),degrees=100,
                      patience=early_stopping_params['patience'])

Ultralytics YOLOv8.2.56 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/Yolov3_runs/runs_100epochs-3-fold/train222/weights/last.pt, data=/content/drive/MyDrive/Combined Dataset3/2024-07-07_5-Fold_Cross-val/split_4/split_4_dataset.yaml, epochs=100, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_ma

100%|██████████| 755k/755k [00:00<00:00, 23.0MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 1]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     37056  ultralytics.nn.modules.block.Bottleneck      [64, 64]                      
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    295680  ultralytics.nn.modules.block.Bottleneck      [128, 128]                    
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  8   4724736  ultralytics.nn.modules.block.Bottleneck      [256, 256]                    
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

100%|██████████| 6.25M/6.25M [00:00<00:00, 111MB/s]
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


AMP: checks passed ✅


...

     52/100        14G     0.7393     0.5484      1.351         38        640: 100%|██████████| 102/102 [01:10<00:00,  1.45it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:05<00:00,  2.37it/s]

                   all        405       1820      0.943      0.852      0.911       0.81


EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 42, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

52 epochs completed in 1.147 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 207.7MB
Optimizer stripped from runs/detect/train/weights/best.pt, 207.7MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.56 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv3 summary (fused): 226 layers, 103,665,011 parameters, 0 gradients, 282.2 GFLOPs


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:06<00:00,  1.86it/s]


                   all        405       1820      0.946      0.841      0.923      0.812
Speed: 0.1ms preprocess, 10.9ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/train


saving results of the training

In [ ]:
# Source and destination paths
source_folder = '/content/runs'
destination_folder = '/content/drive/MyDrive/Yolov3_runs/runs_100epochs-4th-fold'

# Copy the folder and its contents
shutil.copytree(source_folder, destination_folder)

'/content/drive/MyDrive/Yolov3_runs/runs_100epochs-4th-fold'

Code to disconnect runtime after training

In [ ]:
from google.colab import runtime
runtime.unassign()

# Benchmark

Code to calculate memmory usage of the model

In [ ]:
import torch
from ultralytics import YOLO

def get_memory_usage(model, input_size):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    dummy_input = torch.randn(*input_size).to(device)

    # Warm up
    for _ in range(10):
        model(dummy_input)

    # Measure memory usage
    torch.cuda.reset_peak_memory_stats(device)
    with torch.no_grad():
        model(dummy_input)
    mem_used = torch.cuda.max_memory_allocated(device) / (1024 ** 3)  # Convert to GB
    mem_reserved = torch.cuda.max_memory_reserved(device) / (1024 ** 3)  # Convert to GB
    mem_cached = torch.cuda.memory_reserved(device) / (1024 ** 3)  # Convert to GB

    print(f"Memory Used (GB): {mem_used:.2f}")
    print(f"Memory Reserved (GB): {mem_reserved:.2f}")
    print(f"Memory Cached (GB): {mem_cached:.2f}")

# Load a YOLO model
model = YOLO('/content/drive/MyDrive/Yolov3_runs/runs_100epochs-4th-fold/detect/train/weights/best.pt')  # Replace with the appropriate model path

# Define the input size (batch_size, channels, height, width)
input_size = (1, 3, 640, 640)

# Profile the model
model.info()  # Print model summary including memory usage
get_memory_usage(model, input_size)


YOLOv3 summary: 310 layers, 103,693,235 parameters, 0 gradients, 283.0 GFLOPs

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.986340045928955. Dividing input by 255.
0: 640x640 1 invoice, 15.1ms
Speed: 0.2ms preprocess, 15.1ms inference, 702.5ms postprocess per image at shape (1, 3, 640, 640)

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.986340045928955. Dividing input by 255.
0: 640x640 1 invoice, 14.0ms
Speed: 0.0ms preprocess, 14.0ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.986340045928955. Dividing input by 255.
0: 640x640 1 invoice, 14.2ms
Speed: 0.0ms preprocess, 14.2ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.986340045928955. Dividing input by 255.
0: 640x640 1 invoice, 15.0ms
Speed: 0.0ms preprocess, 15.0ms i

Code to calculate inference time of the model

In [ ]:
from ultralytics.utils.benchmarks import benchmark

# Benchmark on GPU
benchmark(model="/content/drive/MyDrive/Yolov3_runs/runs_100epochs-4th-fold/detect/train/weights/best.pt", data="/content/drive/MyDrive/Combined Dataset3/2024-07-07_5-Fold_Cross-val/split_5/split_5_dataset.yaml", half=False, device='cuda:0')

Setup complete ✅ (12 CPUs, 53.0 GB RAM, 40.4/201.2 GB disk)

Benchmarks complete for best.pt on /content/drive/MyDrive/Combined Dataset3/2024-07-07_5-Fold_Cross-val/split_5/split_5_dataset.yaml at imgsz=160 (886.77s)
                   Format Status❔  Size (MB)  metrics/mAP50-95(B)  Inference time (ms/im)     FPS
0                 PyTorch       ✅      198.1               0.6041                   14.13   70.78
1             TorchScript       ✅      395.9               0.5692                    6.26  159.74
2                    ONNX       ✅      395.5               0.5695                   72.45   13.80
3                OpenVINO       ❌        0.0                  NaN                     NaN     NaN
4                TensorRT       ✅      581.9               0.5696                    4.55  219.85
5                  CoreML       ❌        0.0                  NaN                     NaN     NaN
6   TensorFlow SavedModel       ✅      989.0               0.5693                   12.77   78.32

,Format,Status❔,Size (MB),metrics/mAP50-95(B),Inference time (ms/im),FPS
0,PyTorch,✅,198.1,0.6041,14.13,70.78
1,TorchScript,✅,395.9,0.5692,6.26,159.74
2,ONNX,✅,395.5,0.5695,72.45,13.80
3,OpenVINO,❌,0.0,NaN,NaN,NaN
4,TensorRT,✅,581.9,0.5696,4.55,219.85
5,CoreML,❌,0.0,NaN,NaN,NaN
6,TensorFlow SavedModel,✅,989.0,0.5693,12.77,78.32
7,TensorFlow GraphDef,✅,395.6,0.5693,46.54,21.49
8,TensorFlow Lite,❌,0.0,NaN,NaN,NaN
9,TensorFlow Edge TPU,❌,0.0,NaN,NaN,NaN


# Evaluation

Code to calculate performance metrics

In [ ]:

# Load the YOLOv3 model
model = YOLO('/content/drive/MyDrive/Yolov3_runs/runs_100epochs-4th-fold/detect/train/weights/best.pt')
# Path to the dataset configuration file
data_config_path = '/content/drive/MyDrive/Combined Dataset3/2024-07-07_5-Fold_Cross-val/split_5/split_5_dataset.yaml'

# Perform evaluation
results = model.val(data=data_config_path, save_json=True)


Ultralytics YOLOv8.2.68 🚀 Python-3.10.12 torch-2.3.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv3 summary (fused): 226 layers, 103,665,011 parameters, 0 gradients, 282.2 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 14.6MB/s]
val: Scanning /content/drive/MyDrive/Combined Dataset3/2024-07-07_5-Fold_Cross-val/split_5/val/labels.cache... 405 images, 0 backgrounds, 0 corrupt: 100%|██████████| 405/405 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [33:52<00:00, 78.16s/it]


                   all        405       1743       0.94      0.865      0.926      0.822
Speed: 6.6ms preprocess, 4934.7ms inference, 0.0ms loss, 1.5ms postprocess per image
Saving runs/detect/val/predictions.json...
Results saved to runs/detect/val


In [ ]:
results.box.map75

0.8756691825218096

In [ ]:
# Extract metrics from DetMetrics object
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP = results.results_dict['metrics/mAP50(B)']

# If available, extract mAP50-95
if 'metrics/mAP50-95(B)' in results.results_dict:
    mAP50_95 = results.results_dict['metrics/mAP50-95(B)']
else:
    mAP50_95 = None

# Calculate F1 Score (if precision and recall are available)
if precision is not None and recall is not None:
    f1 = 2 * (precision * recall) / (precision + recall)
else:
    f1 = None

# Print collected metrics
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"mAP50: {mAP}")
if mAP50_95 is not None:
    print(f"mAP50-95: {mAP50_95}")
if f1 is not None:
    print(f"F1 Score: {f1}")


Precision: 0.9401661323697852
Recall: 0.8654289449190956
mAP50: 0.9264941173412692
mAP50-95: 0.822415099324223
F1 Score: 0.9012507778955059


In [ ]:
from google.colab import runtime
runtime.unassign()



---

# Experimentation codes

# Without k-fold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.0/793.0 kB 13.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [ ]:
import os
from ultralytics import YOLO

# Load a model
model = YOLO('yolov3.yaml')  # build a new model from scratch

# Define early stopping parameters
early_stopping_params = {
    'patience': 5,  # Number of epochs to wait after last improvement
}

# Use the model with early stopping
results = model.train(data=os.path.join('/content/drive/MyDrive/Combined Dataset2/', 'data.yaml'),
                      patience=early_stopping_params['patience'],)

Ultralytics YOLOv8.2.38 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov3.yaml, data=/content/drive/MyDrive/Combined Dataset2/data.yaml, epochs=100, time=None, patience=5, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tr

100%|██████████| 755k/755k [00:00<00:00, 20.6MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 1]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     37056  ultralytics.nn.modules.block.Bottleneck      [64, 64]                      
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    295680  ultralytics.nn.modules.block.Bottleneck      [128, 128]                    
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  8   4724736  ultralytics.nn.modules.block.Bottleneck      [256, 256]                    
  7                  -1  1   1180672  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 99.5MB/s]


AMP: checks passed ✅


...

     30/100      14.1G     0.5383     0.4492      1.144         62        640: 100%|██████████| 105/105 [01:13<00:00,  1.43it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:04<00:00,  2.40it/s]

                   all        293       1196      0.974      0.784      0.875      0.806
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 25, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



30 epochs completed in 0.700 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 207.7MB
Optimizer stripped from runs/detect/train/weights/best.pt, 207.7MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.38 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv3 summary (fused): 226 layers, 103665011 parameters, 0 gradients, 282.2 GFLOPs


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:05<00:00,  1.81it/s]


                   all        293       1196      0.977       0.79      0.886      0.807
Speed: 0.1ms preprocess, 9.9ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/train


In [ ]:
#saving best run result
import shutil

# Source and destination paths
source_folder = '/content/runs'
destination_folder = '/content/drive/MyDrive/Yolov3_runs/runs_100epochs-30best'

# Copy the folder and its contents
shutil.copytree(source_folder, destination_folder)

'/content/drive/MyDrive/Yolov3_runs/runs_100epochs-30best'

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('/content/drive/MyDrive/Yolov3_runs/runs_100epochs-30best/detect/train/weights/best.pt')  # load a custom model

# Predict with the model
results1 = model('/content/drive/MyDrive/Combined Dataset2/test/images/combined_3.jpg')  # predict on an image

# Process the results (e.g., bounding boxes, masks, keypoints, etc.)
for result in results1:
    boxes = result.boxes
    masks = result.masks
    keypoints = result.keypoints
    probs = result.probs
    obb = result.obb
    result.show()  # Display the result
    result.save(filename='result.jpg')  # Save the result to disk



image 1/1 /content/drive/MyDrive/Combined Dataset2/test/images/combined_3.jpg: 640x448 5 invoices, 3248.5ms
Speed: 3.1ms preprocess, 3248.5ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 448)


In [ ]:
import torch
from ultralytics import YOLO
from tqdm import tqdm
import numpy as np
from sklearn.metrics import precision_recall_curve, average_precision_score, confusion_matrix, f1_score
import time

# Load the YOLOv3 model
model = YOLO('/content/drive/MyDrive/Yolov3_runs/runs_100epochs-30best/detect/train/weights/best.pt')
# Path to the dataset configuration file
data_config_path = '/content/drive/MyDrive/Combined Dataset2/data.yaml'

# Perform evaluation
results = model.val(data=data_config_path, save_json=True)



Ultralytics YOLOv8.2.42 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (AMD EPYC 7B12)
YOLOv3 summary (fused): 226 layers, 103665011 parameters, 0 gradients, 282.2 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 41.4MB/s]
val: Scanning /content/drive/MyDrive/Combined Dataset2/valid/labels.cache... 293 images, 0 backgrounds, 0 corrupt: 100%|██████████| 293/293 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/drive/MyDrive/Combined Dataset2/valid/images/1152-receipt_jpg.rf.eb0a1740575320d9ad8ff88974618c7e.jpg: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [05:08<00:00, 16.24s/it]


                   all        293       1196      0.976      0.791      0.887      0.808
Speed: 1.6ms preprocess, 895.5ms inference, 0.0ms loss, 1.7ms postprocess per image
Saving runs/detect/val3/predictions.json...
Results saved to runs/detect/val3


0it [00:00, ?it/s]


TypeError: 'DetMetrics' object is not iterable

In [ ]:
# Extract metrics from DetMetrics object
precision = results.results_dict['metrics/precision(B)']
recall = results.results_dict['metrics/recall(B)']
mAP = results.results_dict['metrics/mAP50(B)']

# If available, extract mAP50-95
if 'metrics/mAP50-95(B)' in results.results_dict:
    mAP50_95 = results.results_dict['metrics/mAP50-95(B)']
else:
    mAP50_95 = None

# Calculate F1 Score (if precision and recall are available)
if precision is not None and recall is not None:
    f1 = 2 * (precision * recall) / (precision + recall)
else:
    f1 = None

# Print collected metrics
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"mAP50: {mAP}")
if mAP50_95 is not None:
    print(f"mAP50-95: {mAP50_95}")
if f1 is not None:
    print(f"F1 Score: {f1}")

# Measure Inference Time and Memory Usage
inference_times = []
for _ in range(100):
    start_time = time.time()
    model(data_config_path)
    inference_times.append(time.time() - start_time)
average_inference_time = np.mean(inference_times)

memory_usage = torch.cuda.memory_allocated()

print(f"Average Inference Time: {average_inference_time} seconds")
print(f"Memory Usage: {memory_usage} bytes")

# Save metrics to a JSON file for further analysis
import json
metrics = {
    "precision": precision,
    "recall": recall,
    "mAP50": mAP,
    "mAP50-95": mAP50_95,
    "f1_score": f1,
    "average_inference_time": average_inference_time,
    "memory_usage": memory_usage
}

with open('model_metrics.json', 'w') as f:
    json.dump(metrics, f)

Precision: 0.9761673707666685
Recall: 0.7909698996655519
mAP50: 0.887004114996866
mAP50-95: 0.8077889806273552
F1 Score: 0.8738642099074131



FileNotFoundError: No images or videos found in /content/drive/MyDrive/Combined Dataset2/data.yaml. Supported formats are:
images: {'pfm', 'bmp', 'png', 'mpo', 'tif', 'webp', 'jpg', 'jpeg', 'tiff', 'dng'}
videos: {'gif', 'm4v', 'mpg', 'mov', 'asf', 'webm', 'mkv', 'ts', 'wmv', 'mp4', 'avi', 'mpeg'}

In [ ]:
# Measure Inference Time
inference_times = []
for _ in range(100):
    start_time = time.time()
    model.val(data=data_config_path)  # Use val method for validation
    inference_times.append(time.time() - start_time)
average_inference_time = sum(inference_times) / len(inference_times)

print(f"Average Inference Time: {average_inference_time} seconds")

Ultralytics YOLOv8.2.42 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (AMD EPYC 7B12)


val: Scanning /content/drive/MyDrive/Combined Dataset2/valid/labels.cache... 293 images, 0 backgrounds, 0 corrupt: 100%|██████████| 293/293 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/drive/MyDrive/Combined Dataset2/valid/images/1152-receipt_jpg.rf.eb0a1740575320d9ad8ff88974618c7e.jpg: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [04:23<00:00, 13.87s/it]


                   all        293       1196      0.976      0.791      0.887      0.808
Speed: 1.5ms preprocess, 855.2ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/val4
Ultralytics YOLOv8.2.42 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (AMD EPYC 7B12)


...

                   all        293       1196      0.976      0.791      0.887      0.808
Speed: 1.4ms preprocess, 855.2ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/val10
Ultralytics YOLOv8.2.42 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (AMD EPYC 7B12)


val: Scanning /content/drive/MyDrive/Combined Dataset2/valid/labels.cache... 293 images, 0 backgrounds, 0 corrupt: 100%|██████████| 293/293 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/drive/MyDrive/Combined Dataset2/valid/images/1152-receipt_jpg.rf.eb0a1740575320d9ad8ff88974618c7e.jpg: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  32%|███▏      | 6/19 [01:26<03:07, 14.42s/it]


KeyboardInterrupt: 

In [ ]:
# Save metrics to a JSON file for further analysis
import json
metrics = {
    "precision": precision,
    "recall": recall,
    "mAP50": mAP,
    "mAP50-95": mAP50_95,
    "f1_score": f1,
    "average_inference_time": average_inference_time,
    "memory_usage": memory_usage
}

with open('model_metrics.json', 'w') as f:
    json.dump(metrics, f)